In [1]:
RENAME_COLS = {
    'Jobs_1': 'Jobs',
    'Jobs_2': 'Description',
    'Jobs_3': 'Flavour',
}

In [2]:
import pandas as pd
import numpy as np

In [3]:
weapons_df = pd.read_html('https://finalfantasy.fandom.com/wiki/Final_Fantasy_weapons')

In [4]:
fists = weapons_df[1]
daggers = weapons_df[2]
swords = weapons_df[3]
katanas = weapons_df[4]
nunchakus = weapons_df[5]
axes = weapons_df[6]
hammers = weapons_df[7]
staves = weapons_df[8]

In [5]:
weapon_dfs = [
    fists, daggers, swords, katanas,
    nunchakus, axes, hammers, staves,
]
weapon_types = [
    'Fists', 'Daggers', 'Swords', 'Katanas',
    'Nunchakus', 'Axes', 'Hammers', 'Staves',
]

In [6]:
all_weapons_dfs = pd.DataFrame()

for df, weapon_type in zip(weapon_dfs, weapon_types):
    df['Weapon Type'] = weapon_type
    all_weapons_dfs = pd.concat([all_weapons_dfs, df], ignore_index=True)

In [7]:
#Each table row (human observable row) is actually structured
#as 2-3 rows in the website's code with the expection of Fists->Hands
#which occupy only one row
# E.g.
#all_weapons_dfs.head(5)

# If there are 2 rows, the second entry will share the Name of the item
# and the Jobs, Atk, Acc, Crit, and Cost columns will be populated with
# flavour text
# E.g.
# all_weapons_dfs.head(3)

# If there are 3 rows, the second and third entry will share the Name 
# of the item and the Jobs, Atk, Acc, Crit, and Cost columns will 
# be populated with the item description (second entry) or the
# flavour text (third entry)
# E.g.
# all_weapons_dfs.tail(3)

In [8]:
all_weapons_dfs['Count'] = all_weapons_dfs.groupby('Name').cumcount()

In [9]:
reshaped = all_weapons_dfs.pivot(index='Name', columns='Count', values=['Jobs'])

PIVOT_RENAME = {
    '0': 'Jobs',
    '1': 'Effect',
    '2': 'Description',
}

reshaped.columns = [f'{i}' for col, i in reshaped.columns]
reshaped.reset_index(inplace=True)
reshaped = reshaped.rename(columns = PIVOT_RENAME)

reshaped['Effect'], reshaped['Description'] = np.where(reshaped['Description'].isna(), (reshaped['Description'], reshaped['Effect']), (reshaped['Effect'], reshaped['Description']))

reshaped['Effect'] = np.where(reshaped['Effect'].isna(), ('No Effect'), (reshaped['Effect']))

In [10]:
grouped = all_weapons_dfs.groupby('Name').first()

grouped.reset_index(inplace=True)

all_weapons_dfs = grouped.merge(reshaped)

In [11]:
all_weapons_dfs['Cost'] = all_weapons_dfs['Cost'].str.replace('(^[^0-9]+?(.*))', 'N/A', regex=True)
all_weapons_dfs['Jobs'] = all_weapons_dfs['Jobs'].str.replace('Black Wizard', 'BW')
all_weapons_dfs['Jobs'] = all_weapons_dfs['Jobs'].str.replace('White Wizard', 'WW')
all_weapons_dfs['Jobs'] = all_weapons_dfs['Jobs'].str.replace('Red Wizard', 'RW')
all_weapons_dfs['Jobs'] = all_weapons_dfs['Jobs'].str.replace('Knight', 'Kn')
all_weapons_dfs['Jobs'] = all_weapons_dfs['Jobs'].str.replace('Ninja', 'Ni')
all_weapons_dfs['Jobs'] = all_weapons_dfs['Jobs'].str.replace('All jobs but Thief', 'Wa, Kn, Ni, RM, RW, BM, BW, Mo, Ma')
all_weapons_dfs['Jobs'] = all_weapons_dfs['Jobs'].str.replace('All jobs', 'Wa, Kn, Th, Ni, RM, RW, BM, BW, Mo, Ma')

In [12]:
all_weapons_dfs['Alternative Name'] = all_weapons_dfs['Name'].str.extract(
    '(\\(.*)'
)

all_weapons_dfs['Alternative Name'] = all_weapons_dfs['Alternative Name'].str.replace(') (', ', ')
all_weapons_dfs['Alternative Name'] = all_weapons_dfs['Alternative Name'].str.replace('(', '')
all_weapons_dfs['Alternative Name'] = all_weapons_dfs['Alternative Name'].str.replace(')', '')

all_weapons_dfs['Name'] = all_weapons_dfs['Name'].str.replace('(\\(.*)', '', regex=True)


In [13]:
b_df = all_weapons_dfs['Acquirement'].str.extract('(?=Buy: (.*?) (?=[A-Z][a-z]+:))|(?=Buy: (.*))', expand=False)
all_weapons_dfs['Buy'] = np.where(b_df[0].notna(), b_df[0], b_df[1])

f_df = all_weapons_dfs['Acquirement'].str.extract('(?=Find: (.*?) (?=[A-Z][a-z]+:))|(?=Find: (.*))', expand=False)
all_weapons_dfs['Find'] = np.where(f_df[0].notna(), f_df[0], f_df[1])

d_df = all_weapons_dfs['Acquirement'].str.extract('(?=Drop: (.*?) (?=[A-Z][a-z]+:))|(?=Drop: (.*))', expand=False)
all_weapons_dfs['Drop'] = np.where(d_df[0].notna(), d_df[0], d_df[1])

In [14]:
BONUS_AREAS = [
    'Earthgift Shrine', 
    'Hellfire Chasm', 
    'Lifespring Grotto', 
    'Whisperwind Cove',
    'Labyrinth of Time',
]

BONUS_ENCOUNTERS = [
    'Black Goblin',
    'Wild Nakk',
    'Gloom Widow',
    'Desertpede',
    'Catoblepas',
    'Abyss Warm',
    'Skudier',
    'Silver Dragon',
    'Dark Elemental',
    'Devil Hound',
    'Hundlegs'
    'Python',
    'Yellow Ogre',
    'Elm Gigas',
    'Death Elemental',
    'Blue Troll',
    'Poison Naga',
    'Sahagin Queen',
    'Revenant',
    'Black Dragon',
    'Blue Dragon',
    'Dragon Zombie',
    'Green Dragon',
    'Holy Dragon',
    'Red Dragon',
    'White Dragon',
    'Yellow Dragon',
    'Death Manticore',
    'Killer Shark',
    'Dark Eye',
    'Dark Wolf',
    'Knocker',
    'Red Flan',
    'Squidraken'
    'Earth Troll',
    'Sekhret',
    'Rock Gargoyle',
    'Poison Eagle',
    'Reaper',
    'Undergrounder',
    'Bonesnatch',
    'Flood Gigas',
    'Mad Ogre',
    'Devil Wizard',
    'Duel Knight',
    'Unicorn',
    'Prototype',
    'Earth Plant',
    'Bloody Eye',
    'Mage Chimera',
    'Mythril Golem',
    'Yamatano Orochi',
    'Blood Tiger',
    'Pharaoh',
    'Hundlegs',
    'Flare Gigas',
]

BONUS_BOSSES = [
    'Ahriman',
    'Cerberus',
    'Echidna',
    'Two-Headed Dragon',
    'Cognazzo',
    'Scarmiglione',
    'Barbariccia',
    'Rubicante',
    'Gilgamesh',
    'Atomos',
    'Omega',
    'Shinrya',
    'Astos',
    'Vampire',
    'Typhon',
    'Orthros',
    'Phatom Train',
    'Death Gaze',
    'Chronodia',
]

BONUS_CONTENT = BONUS_AREAS + BONUS_ENCOUNTERS + BONUS_BOSSES

all_weapons_dfs['Bonus Content'] = all_weapons_dfs['Acquirement'].apply(lambda x: True if any(bonus in x for bonus in BONUS_CONTENT) else False)

In [15]:
all_weapons_dfs = all_weapons_dfs.fillna('N/A')

In [16]:
split_costs = all_weapons_dfs.copy()

split_costs['Cost (Easy)'] = split_costs['Cost'].str.extract(
    '(.*) \\(Easy\\) ')
split_costs['Cost (Normal)'] = split_costs['Cost'].str.extract(
    '\\(Easy\\) (.*) \\(Normal\\)')

split_costs['Cost (Easy)'] = np.where(split_costs['Cost (Easy)'].isna(),
    split_costs['Cost'], split_costs['Cost (Easy)'])
split_costs['Cost (Normal)'] = np.where(split_costs['Cost (Normal)'].isna(),
    split_costs['Cost'], split_costs['Cost (Normal)'])

all_weapons_dfs = split_costs

In [21]:
max_update = all_weapons_dfs.copy()

max_update['Atk'] = max_update['Atk'].str.replace('Maximum Atk 100', ' (Max Atk 100)')
max_update['Acc'] = max_update['Ac'].str.replace('Maximum Atk 100', ' (Max Atk 100)')

max_update['Atk'].iloc[60:62]

60                             18
61    Current HP/10 (Max Atk 100)
Name: Atk, dtype: object

In [17]:
drop_candidates = [
    'Acquirement', 'Count', 'NES Image', 
    'PS/GBA Image', 'PSP Image', 'PSP/Mobile Image',
]
drop_columns = [x for x in drop_candidates if x in all_weapons_dfs.columns]

all_weapons_dfs = all_weapons_dfs.drop(columns=drop_columns)

In [18]:
all_weapons_dfs.to_csv('ff1_weapons.csv', index=False)